## Merge object classes from different sources into single dataset

In [5]:
cd ../

C:\Users\luisr\Repositories\Flask APIs\ai-camera-client


In [1]:
from ultralytics import YOLO

#### Save yolov8 class names

In [25]:
yolov8_names_path = 'data/yolov8/yolov8.names'

model = YOLO()
results = model.predict()
classes = results[0].names
class_names = [classes[i] for i in sorted(list(classes.keys()))]
with open(yolov8_names_path, 'w') as fw:
    fw.write('\n'.join(class_names))
print('\nYolov8 names saved to:', yolov8_names_path)

WARNING ⚠️ 'source' is missing. Using 'source=C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets'.

image 1/2 C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 122.0ms
image 2/2 C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets\zidane.jpg: 384x640 2 persons, 1 tie, 126.4ms
Speed: 3.0ms preprocess, 124.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Yolov8 names saved to: data/yolov8/yolov8.names


In [6]:
yolov8_names_path = 'data/yolov8/yolov8-fire.names'

model = YOLO('../vision-ai/models/yolov8-fire.pt')
results = model.predict()
classes = results[0].names
class_names = [classes[i] for i in sorted(list(classes.keys()))]
with open(yolov8_names_path, 'w') as fw:
    fw.write('\n'.join(class_names))
print('\nYolov8 names saved to:', yolov8_names_path)

WARNING ⚠️ 'source' is missing. Using 'source=C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets'.

image 1/2 C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets\bus.jpg: 640x480 1 other, 158.2ms
image 2/2 C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets\zidane.jpg: 384x640 1 other, 104.0ms
Speed: 8.1ms preprocess, 131.1ms inference, 7.5ms postprocess per image at shape (1, 3, 384, 640)

Yolov8 names saved to: data/yolov8/yolov8-fire.names


In [6]:
yolov8_names_path = 'data/yolov8/yolov8-gun.names'

model = YOLO('../vision-ai/models/yolov8-gun.pt')
results = model.predict()
classes = results[0].names
class_names = [classes[i] for i in sorted(list(classes.keys()))]
with open(yolov8_names_path, 'w') as fw:
    fw.write('\n'.join(class_names))
print('\nYolov8 names saved to:', yolov8_names_path)

WARNING ⚠️ 'source' is missing. Using 'source=C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets'.

image 1/2 C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets\bus.jpg: 640x480 1 other, 158.2ms
image 2/2 C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets\zidane.jpg: 384x640 1 other, 104.0ms
Speed: 8.1ms preprocess, 131.1ms inference, 7.5ms postprocess per image at shape (1, 3, 384, 640)

Yolov8 names saved to: data/yolov8/yolov8-fire.names


In [7]:
yolov8_names_path = 'data/yolov8/yolov8-gun.names'

model = YOLO('../vision-ai/models/yolov8-gun.pt')
results = model.predict()
classes = results[0].names
class_names = [classes[i] for i in sorted(list(classes.keys()))]
with open(yolov8_names_path, 'w') as fw:
    fw.write('\n'.join(class_names))
print('\nYolov8 names saved to:', yolov8_names_path)

WARNING ⚠️ 'source' is missing. Using 'source=C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets'.

image 1/2 C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets\bus.jpg: 640x480 1 rifle, 535.6ms
image 2/2 C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets\zidane.jpg: 384x640 (no detections), 491.2ms
Speed: 4.0ms preprocess, 513.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

Yolov8 names saved to: data/yolov8/yolov8-gun.names


In [8]:
yolov8_names_path = 'data/yolov8/yolov8-h.names'

model = YOLO('../vision-ai/models/yolov8-helmet.pt')
results = model.predict()
classes = results[0].names
class_names = [classes[i] for i in sorted(list(classes.keys()))]
with open(yolov8_names_path, 'w') as fw:
    fw.write('\n'.join(class_names))
print('\nYolov8 names saved to:', yolov8_names_path)

WARNING ⚠️ 'source' is missing. Using 'source=C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets'.

image 1/2 C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets\bus.jpg: 640x480 3 - HelmetDetectionImageDataset - 2023-10-19 7-15ams, 549.1ms
image 2/2 C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets\zidane.jpg: 384x640 2 - HelmetDetectionImageDataset - 2023-10-19 7-15ams, 440.5ms
Speed: 3.5ms preprocess, 494.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

Yolov8 names saved to: data/yolov8/yolov8-helmet.names


In [ ]:
yolov8_names_path = 'data/yolov8/yolov8-helmet.names'

model = YOLO('../vision-ai/models/yolov8-helmet.pt')
results = model.predict()
classes = results[0].names
class_names = [classes[i] for i in sorted(list(classes.keys()))]
with open(yolov8_names_path, 'w') as fw:
    fw.write('\n'.join(class_names))
print('\nYolov8 names saved to:', yolov8_names_path)

WARNING ⚠️ 'source' is missing. Using 'source=C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets'.


#### Save custom yolov8 handsup detection model

In [20]:
yolov8_names_path = 'data/yolov8/yolov8-handsup.names'

class_names = ['hands_up', 'left_hand_up', 'right_hand_up']

with open(yolov8_names_path, 'w') as fw:
    fw.write('\n'.join(class_names))
    
print('\nYolov8 Hands Up names saved to:', yolov8_names_path)


Yolov8 Hands Up names saved to: data/yolov8/yolov8-handsup.names


##### Move to Vision AI API directory

In [10]:
cd ../vision-ai/

C:\Users\luisr\Repositories\Flask APIs\vision-ai


#### Save yolov5 class names

In [17]:
yolov5_names_path = '../ai-camera-client/data/yolov5/yolov5-objects365.names'

import cv2
from modules.yolov5_wrap import YOLOV5Wrap

image = cv2.imread('nbs/img.jpg')

model = YOLOV5Wrap(model_name='objects365', path='models/yolov5m_Objects365.pt') # 'yolov5s' # or 'objects365'
classes = model.model.names
class_names = [classes[i] for i in sorted(list(classes.keys()))]
with open(yolov5_names_path, 'w') as fw:
    fw.write('\n'.join(class_names))

print('\nYolov5 Objects365 names saved to:', yolov5_names_path)

YOLOv5  2024-4-20 Python-3.11.7 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 22323858 parameters, 0 gradients, 52.6 GFLOPs
Adding AutoShape... 



Yolov5 Objects365 names saved to: ../ai-camera-client/data/yolov5/yolov5-objects365.names


##### Move back to AI Camera Client API directory

In [19]:
cd ../ai-camera-client/

C:\Users\luisr\Repositories\Flask APIs\ai-camera-client


#### Construct model datasets

In [51]:
model_dataset = [{
    'names_path': 'data/yolov8/yolov8-fire.names',
    'model_path': 'models/yolov8-fire.pt',
    'model_name': 'yolov8-fire',
    'model_class': 'YOLOWrap',
}, {
    'names_path': 'data/yolov8/yolov8-gun.names',
    'model_path': 'models/yolov8-gun.pt',
    'model_name': 'yolov8-gun',
    'model_class': 'YOLOWrap',
}, {
    'names_path': 'data/yolov8/yolov8-handsup.names',
    'model_path': 'models/yolov8s-pose.pt',
    'model_name': 'yolov8-handsup',
    'model_class': 'YoloV8HandsUp',
}, {
    'names_path': 'data/yolov8/yolov8-helmet.names',
    'model_path': 'models/yolov8-helmet.pt',
    'model_name': 'yolov8-helmet',
    'model_class': 'YOLOWrap',
}, {
    'names_path': 'data/yolov8/yolov8.names',
    'model_path': 'models/yolov8s.pt',
    'model_name': 'yolov8',
    'model_class': 'YOLO',
}, {
    'names_path': 'data/yolov5/yolov5-objects365.names',
    'model_path': 'models/yolov5m_Objects365.pt',
    'model_name': 'yolov5-objects365',
    'model_class': 'YOLOV5Wrap',
}, {
    'names_path': 'data/yolov4/yolov4.names',
    'model_path': 'models/yolov4.json',
    'model_name': 'yolov4',
    'model_class': 'YoloV4Wrap',
}]

#### Merge object datasets for each model

In [52]:
object_dataset = []

for i, row in enumerate(model_dataset):
    names = open(row['names_path']).read().split('\n')

    objects = [{**row, 'class_id': i, 'class_name': name} for i, name in enumerate(names)]

    object_dataset.extend(objects)

---
### Manage mongo database

#### Set connection parameters

In [55]:
MONGO_API_URL = 'http://localhost:5000'
db = 'octacity'
coll = 'classes'

#### Get mongo records

In [56]:
import requests

url = f'{MONGO_API_URL}/{db}/{coll}'

res = requests.get(url)

print('status:', res.status_code)
print('message:', res.reason)

if res.ok:
    objects_mongo = res.json()
    print('\n', objects_mongo[:5])

status: 200
message: OK

 [{'_id': '66358a4d6b6e1bcc3d4788fc', 'class_id': 0, 'class_name': 'person', 'model_class': 'YOLO', 'model_name': 'yolov8', 'model_path': 'models/yolov8s.pt', 'names_path': 'data/yolov8/yolov8.names'}, {'_id': '66358a4d6b6e1bcc3d4788fd', 'class_id': 1, 'class_name': 'bicycle', 'model_class': 'YOLO', 'model_name': 'yolov8', 'model_path': 'models/yolov8s.pt', 'names_path': 'data/yolov8/yolov8.names'}, {'_id': '66358a4d6b6e1bcc3d4788fe', 'class_id': 2, 'class_name': 'car', 'model_class': 'YOLO', 'model_name': 'yolov8', 'model_path': 'models/yolov8s.pt', 'names_path': 'data/yolov8/yolov8.names'}, {'_id': '66358a4d6b6e1bcc3d4788ff', 'class_id': 3, 'class_name': 'motorcycle', 'model_class': 'YOLO', 'model_name': 'yolov8', 'model_path': 'models/yolov8s.pt', 'names_path': 'data/yolov8/yolov8.names'}, {'_id': '66358a4d6b6e1bcc3d478900', 'class_id': 4, 'class_name': 'airplane', 'model_class': 'YOLO', 'model_name': 'yolov8', 'model_path': 'models/yolov8s.pt', 'names_path'

#### Delte all records in mongo collection

In [57]:
import requests

url = f'{MONGO_API_URL}/{db}/{coll}'

res = requests.delete(url)

print('status:', res.status_code)
print('message:', res.reason)
print('response:', res.text)

status: 200
message: OK
response: {"deleted_count":536,"message":"All records deleted successfully"}



#### Post objects to mongo collection

In [58]:
import requests

url = f'{MONGO_API_URL}/{db}/{coll}'
headers = {'Content-Type': 'application/json'}
data = object_dataset.copy()

res = requests.post(url, headers=headers, json=data)

print('status:', res.status_code)
print('message:', res.reason)
print('response:', res.text)

status: 201
message: CREATED
response: {"data":[{"_id":"66359433f80018829b662dba","class_id":0,"class_name":"fire","model_class":"YOLOWrap","model_name":"yolov8-fire","model_path":"models/yolov8-fire.pt","names_path":"data/yolov8/yolov8-fire.names"},{"_id":"66359433f80018829b662dbb","class_id":1,"class_name":"other","model_class":"YOLOWrap","model_name":"yolov8-fire","model_path":"models/yolov8-fire.pt","names_path":"data/yolov8/yolov8-fire.names"},{"_id":"66359433f80018829b662dbc","class_id":2,"class_name":"smoke","model_class":"YOLOWrap","model_name":"yolov8-fire","model_path":"models/yolov8-fire.pt","names_path":"data/yolov8/yolov8-fire.names"},{"_id":"66359433f80018829b662dbd","class_id":0,"class_name":"handgun","model_class":"YOLOWrap","model_name":"yolov8-gun","model_path":"models/yolov8-gun.pt","names_path":"data/yolov8/yolov8-gun.names"},{"_id":"66359433f80018829b662dbe","class_id":1,"class_name":"rifle","model_class":"YOLOWrap","model_name":"yolov8-gun","model_path":"models/yo